In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Evaluate Generated Answers from Retrieval-Augmented Generation (RAG) for Question Answering with Rapid Evaluation SDK

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluate_rag_rapid_evaluation_sdk.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fevaluation%2Fevaluate_rag_rapid_evaluation_sdk.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluate_rag_rapid_evaluation_sdk.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/evaluation/evaluate_rag_rapid_evaluation_sdk.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>    
</table>

| | |
|-|-|
|Author(s) | [Jason Dai](https://github.com/jsondai) |

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview


In this tutorial, you will learn how to use the use the *Vertex AI Python SDK for Rapid Evaluation* to evaluate **Retrieval-Augmented Generation** (RAG) generated answers for **Question Answering** (QA) task.

RAG is a technique to improve groundness, relevancy and factuality of large language models (LLMs) by finding relevant information from the model's knowledge base. RAG is done by converting a query into a vector representation (embeddings), and then finding the most similar vectors in the knowledge base. The most similar vectors are then used to help generate the response.

This tutorial demonstrates how to use Rapid Evaluation for a Bring-Your-Own-Answer scenario:

* The context were retrieved and answers were generated based on the retrieved context using RAG.

* Evaluate the quality of the RAG generated answers for QA task programmatically using the SDK.

The examples used in this notebook is from Stanford Question Answering Dataset [SQuAD 2.0](https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1194/reports/default/15785042.pdf).

Learn more about [Vertex AI Rapid Evaluation SDK](https://cloud.google.com/vertex-ai/generative-ai/docs/models/online-pipeline-services).


## Getting Started

### Install Vertex AI SDK for Rapid Evaluation

In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform[rapid_evaluation]
%pip install --quiet --upgrade nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 12.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Authenticate your notebook environment (Colab only)

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

In [4]:
PROJECT_ID = "<project id>"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [5]:
# General
from IPython.display import display, Markdown, HTML
import logging
import nest_asyncio
import warnings

# Main
from vertexai.preview.evaluation import EvalTask
import pandas as pd

### Library settings

In [6]:
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
nest_asyncio.apply()
warnings.filterwarnings("ignore")

### Helper functions

In [7]:
def display_eval_report(eval_result, metrics=None):
    """Display the evaluation results."""

    title, summary_metrics, report_df = eval_result
    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient="index").T
    if metrics:
        metrics_df = metrics_df.filter(
            [
                metric
                for metric in metrics_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )
        report_df = report_df.filter(
            [
                metric
                for metric in report_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    # Display the title with Markdown for emphasis
    display(Markdown(f"## {title}"))

    # Display the metrics DataFrame
    display(Markdown("### Summary Metrics"))
    display(metrics_df)

    # Display the detailed report DataFrame
    display(Markdown(f"### Report Metrics"))
    display(report_df)


def display_explanations(df, metrics=None, n=1):
    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = df.sample(n=n)
    if metrics:
        df = df.filter(
            ["instruction", "context", "reference", "completed_prompt", "response"]
            + [
                metric
                for metric in df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    for index, row in df.iterrows():
        for col in df.columns:
            display(HTML(f"<h2>{col}:</h2> <div style='{style}'>{row[col]}</div>"))
        display(HTML("<hr>"))

# Bring-Your-Own-Answer Evaluation for RAG

Perform bring-your-own-answer evaluation by assessing the generated answer's quality based on the context retrieved or the reference provided.

## Prepare Dataset

To evaluate the RAG generated answers, the evaluation dataset is required to contain the following fields:

* Question
* Context
* RAG Generated Answer

In [8]:
questions = [
    "Which part of the brain does short-term memory seem to rely on?",
    "What provided the Roman senate with exuberance?",
    "What area did the Hasan-jalalians command?",
]

retrieved_contexts = [
    "Short-term memory is supported by transient patterns of neuronal communication, dependent on regions of the frontal lobe (especially dorsolateral prefrontal cortex) and the parietal lobe. Long-term memory, on the other hand, is maintained by more stable and permanent changes in neural connections widely spread throughout the brain. The hippocampus is essential (for learning new information) to the consolidation of information from short-term to long-term memory, although it does not seem to store information itself. Without the hippocampus, new memories are unable to be stored into long-term memory, as learned from patient Henry Molaison after removal of both his hippocampi, and there will be a very short attention span. Furthermore, it may be involved in changing neural connections for a period of three months or more after the initial learning.",
    "In 62 BC, Pompey returned victorious from Asia. The Senate, elated by its successes against Catiline, refused to ratify the arrangements that Pompey had made. Pompey, in effect, became powerless. Thus, when Julius Caesar returned from a governorship in Spain in 61 BC, he found it easy to make an arrangement with Pompey. Caesar and Pompey, along with Crassus, established a private agreement, now known as the First Triumvirate. Under the agreement, Pompey's arrangements would be ratified. Caesar would be elected consul in 59 BC, and would then serve as governor of Gaul for five years. Crassus was promised a future consulship.",
    "The Seljuk Empire soon started to collapse. In the early 12th century, Armenian princes of the Zakarid noble family drove out the Seljuk Turks and established a semi-independent Armenian principality in Northern and Eastern Armenia, known as Zakarid Armenia, which lasted under the patronage of the Georgian Kingdom. The noble family of Orbelians shared control with the Zakarids in various parts of the country, especially in Syunik and Vayots Dzor, while the Armenian family of Hasan-Jalalians controlled provinces of Artsakh and Utik as the Kingdom of Artsakh.",
]

generated_answers = [
    "frontal lobe and the parietal lobe",
    "The Roman Senate was filled with exuberance due to successes against Catiline.",
    "The Hasan-Jalalians commanded the area of Syunik and Vayots Dzor.",
]


eval_dataset = pd.DataFrame(
    {
        "instruction": questions,
        "context": retrieved_contexts,
        "response": generated_answers,
    }
)

## Select Metric and Define EvalTask


Choose from the following metrics for an evaluation task. For more information about the supported evaluation metrics and how to use each metric, please see [Evaluation methods and metrics](https://cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval).

*   `question_answering_quality` (overall quality)
*   `question_answering_relevance`
*   `question_answering_helpfulness`
*   `groundedness`
*   `fulfillment`

You can run evaluation for just one metric, or a combination of metrics. For example, we create an `EvalTask` named `answer_eval_task` with all the QA-related metrics to compute all the metrics in one eval run as follows:

In [9]:
answer_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[
        "question_answering_quality",
        "question_answering_relevance",
        "question_answering_helpfulness",
        "groundedness",
        "fulfillment",
    ],
    experiment="rag-eval-01",
)

## Run Evaluation

In [12]:
# Retry the evaluation
result = answer_eval_task.evaluate()

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/474775107710/locations/us-central1/metadataStores/default/contexts/rag-eval-01-3593551d-4e12-4908-8e8c-fd47c93a19f0 to Experiment: rag-eval-01
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 15 Vertex online evaluation service requests.
INFO:vertexai.preview.evaluation._evaluation:All 15 metrics are successfully computed.


## Display Evaluation Result

### Overall Evaluation Result

If you want to have an overall view of all the metrics evaluation result in one table, you can use the `display_eval_report()` helper function.

In [13]:
display_eval_report((("Eval Result", result.summary_metrics, result.metrics_table)))

## Eval Result

### Summary Metrics

,row_count,question_answering_quality/mean,question_answering_quality/std,question_answering_relevance/mean,question_answering_relevance/std,question_answering_helpfulness/mean,question_answering_helpfulness/std,groundedness/mean,groundedness/std,fulfillment/mean,fulfillment/std
0,3.0,3.666667,2.309401,3.666667,2.309401,3.333333,2.081666,0.666667,0.57735,5.0,0.0


### Report Metrics

,instruction,context,response,question_answering_quality/explanation,question_answering_quality/confidence,question_answering_quality,question_answering_relevance/explanation,question_answering_relevance/confidence,question_answering_relevance,question_answering_helpfulness/explanation,question_answering_helpfulness/confidence,question_answering_helpfulness,groundedness/explanation,groundedness/confidence,groundedness,fulfillment/explanation,fulfillment/confidence,fulfillment
0,Which part of the brain does short-term memory...,Short-term memory is supported by transient pa...,frontal lobe and the parietal lobe,The response correctly states that short-term ...,1.0,5.0,The response directly addresses the instructio...,1.0,5.0,The response provides specific and clearly def...,1.0,5.0,"The response ""frontal lobe and the parietal lo...",1.0,1.0,The response correctly identifies two parts of...,0.8,5.0
1,What provided the Roman senate with exuberance?,"In 62 BC, Pompey returned victorious from Asia...",The Roman Senate was filled with exuberance du...,The response is grounded in the context as it ...,1.0,5.0,The response directly addresses the instructio...,1.0,5.0,The response provides a concise and accurate a...,1.0,4.0,The response correctly states that the Roman S...,1.0,1.0,The response directly answers the question by ...,1.0,5.0
2,What area did the Hasan-jalalians command?,The Seljuk Empire soon started to collapse. In...,The Hasan-Jalalians commanded the area of Syun...,The response incorrectly states that the Hasan...,1.0,1.0,The response is not relevant to the instructio...,1.0,1.0,The response is not helpful because it does no...,0.9,1.0,The response states that the Hasan-Jalalians c...,1.0,0.0,The response directly answers the question by ...,1.0,5.0


#### Detailed Explanation for an Individual Instance

If you need to delve into the individual result's detailed explanations on why a score is assigned and how confident the model is for each model-based metric, you can use the `display_explanations()` helper function. For example, you can set `n=2` to display explanation of the 2nd instance result as follows:

In [14]:
display_explanations(result.metrics_table, n=2)

You can also focus on one or a few metrics as follows.

In [15]:
display_explanations(result.metrics_table, metrics=["question_answering_quality"])

**Bring-Your-Own-Answer Evaluation for RAG**
To improve the quality of the RAG System response, one option is to rephrase the user question to improve its clarity and make it easier to understand.
You will use 2 metrics to evaluate this task: Coherence and Fluency.
According to Vertex AI documentation, here is a brief description of both metrics.

Coherence: The coherence metric describes the model's ability to provide a coherent response.
Evaluation criteria for coherence:

Follows logical flow: Ideas logically progress with clear transitions that are relevant to the main point.
Organized: Writing structure is clear, employing topic sentences where appropriate and effective transitions to guide the reader.
Cohesive: Word choices, sentence structures, pronouns, and figurative language reinforce connections between ideas.
Fluency: The fluency metric describes the model's language mastery.
Evaluation criteria for fluency:

Has proper grammar: The language's grammar rules are correctly followed, including but not limited to sentence structures, verb tenses, subject-verb agreement, proper punctuation, and capitalization.
Chooses words appropriately: Words chosen are appropriate and purposeful given their relative context and positioning in the text. The vocabulary demonstrates prompt understanding.
Smooth: Sentences flow smoothly and avoid awkward phrasing or run-on sentences. Ideas and sentences connect logically, using transitions effectively where needed.
Reference: https://cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval

Prepare Dataset
To evaluate the coherence and fluency, simply provide the input questions to the Vertex AI Rapid Evaluation SDK

In [18]:
questions = [
    "Can I configure certificates manually?",
    "How many control plane instances should I use?",
    "Is it possible to run different replicas of a StatefulSet in different zones?",
]

rephrase_dataset = pd.DataFrame(
    {
        "response": questions,
    }
)


Create an EvalTask and define the metrics you want to use.
You can also set an experiment ID to log all the results to Vertex AI Experiments.

In [20]:
eval_rephrase_task = EvalTask(
    dataset=rephrase_dataset,
    metrics=[
        "coherence",
        "fluency"
    ],
    experiment="evaluate-rephrase-01",
)

In [21]:
# Start the evaluation process. Depending on the amount of samples in your evaluation
# dataset, this can take a few minutes to complete.
result = eval_rephrase_task.evaluate()

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/474775107710/locations/us-central1/metadataStores/default/contexts/evaluate-rephrase-01-1b3d535d-756d-4af8-90a5-1bd62a84f303 to Experiment: evaluate-rephrase-01
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 6 Vertex online evaluation service requests.
INFO:vertexai.preview.evaluation._evaluation:All 6 metrics are successfully computed.


**Overall Evaluation Result**

If you want to have an overall view of all the metrics evaluation result in one table, you can use the display_eval_report() helper function.

In [22]:
display_eval_report((("Eval Result", result.summary_metrics, result.metrics_table)))

## Eval Result

### Summary Metrics

,row_count,coherence/mean,coherence/std,fluency/mean,fluency/std
0,3.0,3.0,1.732051,3.666667,1.527525


### Report Metrics

,response,coherence/explanation,coherence/confidence,coherence,fluency/explanation,fluency/confidence,fluency
0,Can I configure certificates manually?,The response lacks a clear topic sentence and ...,1.0,2.0,The sentence is grammatically correct and has ...,0.9,5.0
1,How many control plane instances should I use?,The response is about the number of control pl...,0.8,2.0,The response lacks proper sentence structure a...,0.5,2.0
2,Is it possible to run different replicas of a ...,The response is about the possibility of runni...,1.0,5.0,The sentence has no grammatical errors and is ...,0.4,4.0


**Detailed Explanation for an Individual Instance**

If you need to delve into the individual result's detailed explanations on why a score is assigned and how confident the model is for each model-based metric, you can use the display_explanations() helper function.
For example, you can set n=2 to display explanation of the 2nd instance result as follows:

In [23]:
display_explanations(result.metrics_table, n=2)